## Clasificación de textos utilizando un **ensemble** de clasificadores

La clasificación de textos consiste en, dado un texto, asignarle una entre varias categorías. Algunos ejemplos de esta tarea son:

- dado un tweet, categorizar su connotación como positiva, negativa o neutra.
- dado un post de Facebook, clasificarlo como portador de un lenguaje ofensivo o no.  

En la actividad exploraremos cómo implementar la técnica de **stacking** para combinar modelos y su aplicación para clasificar reviews de [IMDB](https://www.imdb.com/) sobre películas en las categorías \[$positive$, $negative$\]. 



Esta consiste en entrenar un modelo que realiza la clasificación a partir de las predicciones realizadas por otros clasificadores.

Concretamente, combinaremos 
un clasificador incluido en la librería [Transformers](https://huggingface.co/transformers/)  y un pipeline basado Support Vector Machines. Puede encontrar más información sobre los datos utilizados en [Kaggle](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) y en [Large Movie Review Datase](http://ai.stanford.edu/~amaas/data/sentiment/).

**Instrucciones:**

- siga las indicaciones y comentarios en cada apartado.


**Después de esta actividad nos habremos familiarizado con:**
- cómo combinar varios modelos para obtener un clasificador más robusto mediante **stacking**.

- cómo contruir un pipeline para la clasificación de textos utilizando [scikit-learn](https://scikit-learn.org/stable/)..


- cómo instanciar un pipeline para la clasificación de textos utilizando la librería Transformers.

**Requerimientos**
- python 3.6.12 - 3.8
- tensorflow==2.3.0
- transformers==4.2.1
- pandas==1.1.5
- plotly==4.13.0
- tqdm==4.56.0
- scikit-learn==0.24.0

### Instalación de librerías e importación de dependencias.

Para comenzar, es preciso instalar las dependencias, realizar los imports necesarios y definir algunas funciones auxiliares.

Ejecute las siguientes casillas prestando atención a las instrucciones adicionales en los comentarios.

In [1]:
# instalar librerías. Esta casilla es últil por ejemplo si se ejecuta el cuaderno en Google Colab
# Note que existen otras dependencias como tensorflow==2.3.0, etc. que en este caso se encontrarían ya instaladas
%%capture
!pip install transformers==4.2.1

print('Done!')

In [57]:
# para establecer caminos al guardar y leer archivos
import os

#  para construir gráficas y realizar análisis exploratorio de los datos
import plotly.graph_objects as go
from tqdm import tqdm

# para cargar datos y realizar pre-procesamiento básico
import numpy as np
import pandas as pd
from collections import Counter

# para pre-procesamiento del texto y extraer carácterísticas
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem.snowball import EnglishStemmer
from sklearn import preprocessing

# algoritmos de clasificación
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# para construir pipelines
from sklearn.pipeline import Pipeline

# para evaluar los modelos 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, plot_roc_curve
from sklearn.utils.multiclass import unique_labels

# para guardar el modelo
import pickle
import tensorflow as tf

# Comunes


# Modelo 01


# Modelo 02

# algoritmos de clasificación, tokenizadores, etc.
from transformers import TextClassificationPipeline, DistilBertTokenizer, TFDistilBertForSequenceClassification, ModelCard

from transformers.tokenization_utils import TruncationStrategy


# Modelo 03
# algoritmos de clasificación, tokenizadores, etc.
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, ModelCard, DistilBertConfig, TextClassificationPipeline


print('Done!')

Done!


In [3]:
# evalua el pipeline entrenado de acuerdo a una de las métricas apropiadas para un problema de clasificación
def evaluate_model(model, X, y_true):
    y_pred = predict(model, X)

    print('==== Sumario de la clasificación ==== ')
    print(classification_report(y_true, y_pred))
    
    print('Accuracy -> {:.2%}\n'.format(accuracy_score(y_true, y_pred)))
    
    if hasattr(model, 'predict_proba'):
      y_scores = predict_proba(model, X)[:,1]
      rocs = roc_auc_score(y_true, y_scores)
      #rocc = roc_curve(y_true, y_score[:,1], pos_label='positive')

      print('ROC Score ->  {:.2%}\n'.format(rocs))
      plot_roc_curve(model, X, y_true)
      #print(rocc)

    print('==== Matriz de confusión ==== ')
    cm = confusion_matrix(y_true, y_pred)
    display_labels = unique_labels(y_true, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=display_labels)
    disp.plot(include_values=True)

### Carga de datos y análisis exploratorio

El primer paso consiste en obtener los datos relacionados con nuestra tarea dejándolos en el formato adecuado.  Existen diferentes opciones, entre estas:

- montar nuestra partición de Google Drive y leer un fichero desde esta.

- leer los datos desde un fichero en una carpeta local.

- leer los datos directamente de un URL.

En este caso, se encuentran en un fichero separado por comas con la siguiente estructura:

| Phrase | Sentiment| 
| ------ | ------ |
| This movie is really not all that bad...    | positive |


Ejecute la siguiente casilla para leer los datos.

In [4]:
# descomente las siguientes 3 líneas para leer datos desde Google Drive,sumiendo que se trata de un fichero llamado review.csv localizado dentro de una carpeta llamada 'Datos' en su Google Drive
#from google.colab import drive
#drive.mount('/content/drive')
#path = '/content/drive/MyDrive/Datos/ejemplo_review_train.csv'


# descomente la siguiente línea para leer los datos desde un archivo local, por ejemplo, asumiendo que se encuentra dentro de un directorio llamado sample_data
#path = './sample_data/ejemplo_review_train.csv'


# descomente la siguiente línea para leer datos desde un URL
path = 'https://github.com/TeachingTextMining/TextClassification/raw/main/02-SA-Transformers-Basic/sample_data/ejemplo_review_train.csv'


# leer los datos
data = pd.read_csv(path, sep=',')

print('Done!')

Done!


Una vez leídos los datos, ejecute la siguiente casilla para construir una gráfica que muestra la distribución de clases en el corpus.

In [5]:
# obtener algunas estadísticas sobre los datos
categories = sorted(data['Sentiment'].unique(), reverse=True)
hist= Counter(data['Sentiment']) 
print('Total de instancias -> {0}'.format(data.shape[0]))
print('Distribución de clases -> {0}'.format({item[0]:round(item[1]/len(data['Sentiment']), 3) for item in sorted(hist.items(), key=lambda x: x[0])}))

print('Categorías -> {0}'.format(categories))
print('Comentario de ejemplo -> {0}'.format(data['Phrase'][0]))
print('Categoría del comentario -> {0}'.format(data['Sentiment'][0]))

colors = ['darkgreen', 'red']
fig = go.Figure(layout=go.Layout(height=400, width=600))
fig.add_trace(go.Bar(x=categories, y=[hist[cat] for cat in sorted(hist.keys())], marker_color=colors))
fig.show()

print('Done!')

Total de instancias -> 1763
Distribución de clases -> {'negative': 0.511, 'positive': 0.489}
Categorías -> ['positive', 'negative']
Comentario de ejemplo -> This is a great movie that everyone should see. It plays like a Dean Koontz book.<br /><br />Bill Paxton's performance was great in that it really seems like he believes in what he is saying and doing.<br /><br />I don't know why viewers have to read in some kind of advocacy for religious murder in to the film. It is fiction. The ending is surprising, but fictional. So what? I think that is what makes this movie so good. SPOILER DO NOT READ FURTHER IF YOU HAVENT SEEN THE MOVIE. Throughout the movie, the viewer is continually shocked at the sickness of Paxton's character, the impact on the children, and the way the children handle this outrageous conduct. And then at the end, it turns out to be true. God has put him on a mission to rid the world of demons. Paxton is not clairvoyant as other viewers suggest. Sure, he is given info th

Done!


Finalmente, ejecute la siguiente casilla para crear los conjuntos de entrenamiento y validación que se utilizarán para entrenar y validar los modelos.

In [6]:
# obtener conjuntos de entrenamiento (90%) y validación (10%)
seed = 0    # fijar random_state para reproducibilidad
train, val = train_test_split(data, test_size=.1, stratify=data['Sentiment'], random_state=seed)

### Entrenamiento de los clasificadores base

#### Modelo 1: Text Classification Pipeline
El primer modelo será un pipeline básico para la clasificación de textos.

Ejecute la siguiente casilla para definir algunas variables y funciones auxiliares, instanciar el modelo y entrenarlo. Preste atención a las explicaciónes dadas en los comentarios.

In [11]:
# listado de stopwords
stop_words=['i','me','my','myself','we','our','ours','ourselves','you','your','yours','yourself','yourselves',
            'he','him','his','himself','she','her','hers','herself','it','its','itself','they','them','their',
            'theirs','themselves','what','which','who','whom','this','that','these','those','am','is','are',
            'was','were','be','been','being','have','has','had','having','do','does','did','doing','a','an',
            'the','and','but','if','or','because','as','until','while','of','at','by','for','with','about',
            'against','between','into','through','during','before','after','above','below','to','from','up',
            'down','in','out','on','off','over','under','again','further','then','once','here','there','when',
            'where','why','how','all','any','both','each','few','more','most','other','some','such','no','nor',
            'not','only','own','same','so','than','too','very','s','t','can','will','just','don','should','now', 'ever']



# función auxiliar. Se utiliza al obtener la representación mediante TF-IDF del texto pues en este caso
# se removerán las stop_words y se considerarán los "stem" en lugar de las palabras.
def english_stemmer(sentence):
  stemmer = EnglishStemmer() 
  analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=stop_words, ngram_range=(1, 1)).build_analyzer() 
  return (stemmer.stem(word) for word in analyzer(sentence))


# crear el pipeline
model01 = Pipeline([
            ('dataVect', CountVectorizer(analyzer=english_stemmer)),
            ('tfidf', TfidfTransformer(smooth_idf=True, use_idf=True)),
            ('classifier', SVC(probability=True))
          ])


# entrenar el modelo
model01.fit(train['Phrase'], train['Sentiment'])


print('Done!')

Done!


#### Modelo 2: Transformer out-of-the-box sentiment analysis model.

El segundo modelo considerado es un pipeline para la clasificación de textos incluido en la librería Transformers.

Ejecute la siguiente casilla para instanciar el pipeline. Note que no es necesario entrenar pues este paso ya se ha realizado por Transformers.

In [22]:
# configuraciones
cfg02 = {}
cfg02['framework'] = 'tf'
cfg02['task'] = 'sentiment-analysis'
cfg02['trained_model_name'] = 'distilbert-base-uncased-finetuned-sst-2-english'
cfg02['max_length'] = 512    # máxima longitud de secuencia recomendada por DistilBERT
cfg02['truncation'] = TruncationStrategy.ONLY_FIRST

# cargar el tokenizador, disponible en Transformers. Establecer model_max_length para cuando el tokenizador sea llamado, trunque automáticamente.
cfg02['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg02['trained_model_name'] , model_max_length=cfg02['max_length'])


# cargar el modelo, disponible en Transformers
cfg02['transformer'] = TFDistilBertForSequenceClassification.from_pretrained(cfg02['trained_model_name'])
cfg02['modelcard'] = ModelCard.from_pretrained(cfg02['trained_model_name'])

# instanciar el pipeline para la clasificación de textos
model02 = TextClassificationPipeline(model=cfg02['transformer'], tokenizer=cfg02['tokenizer'], modelcard=None, framework=cfg02['framework'], task=cfg02['task'], return_all_scores=False)

print('Done!')

Done!


Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done!


#### Modelo 3: Fine-tunned Transformer (DistilBERT)

El tercer y último clasificador será un modelo basado en Transformers, entrenado específicamente en nuestros datos.

En este caso, es necesario tokenizar y convertir a tensores los datos de acuerdo a los requisitos de Transformers.

Ejecute las siguientes casillas para preprocesar los datos, instanciar el modelo y entrenarlo. Preste atención a las explicaciónes dadas en los comentarios.

In [13]:
# Preprocesamiento de los datos


# configuraciones
cfg03 = {} # diccinario para agrupar configuraciones y variables para su posterior uso
cfg03['framework'] = 'tf'    # TensorFlow como framework (por cuestiones del formato en los datos)
cfg03['max_length'] = 512    # máxima longitud de secuencia recomendada por DistilBERT
cfg03['trained_model_name'] = 'distilbert-base-uncased'



# cargar el tokenizador, disponible en Transformers
cfg03['tokenizer'] = DistilBertTokenizer.from_pretrained(cfg03['trained_model_name'] )
tokenizer = cfg03['tokenizer']

# obtener ids y máscaras para el conjunto de entrenamiento
# no es necesario convertir a tensores porque la salida del tokenizador se encuentra en este formato, 
# ej. ver train_encodings['input_ids'] y train_encodings['attention_mask'] 
train_encodings = tokenizer(train['Phrase'].to_list(), truncation=True, padding='max_length', max_length=cfg03['max_length'], return_tensors=cfg03['framework']) 


# instanciar y entrenar LabelBinarizer
lb = preprocessing.LabelBinarizer()
lb.fit(train['Sentiment'])
num_labels = len(lb.classes_) # variable necesaria para la configuración del modelo, aunque al tratarse de clasificación binaria, se hará 1
cfg03['num_labels'] = 1

# obtener codificación one-hot
train_blabels = lb.transform(train['Sentiment'])


# obtener tensores correspondientes
train_blabels_t = tf.convert_to_tensor(train_blabels, dtype='int32')

print('Done!')

Done!


Una vez preprocesados los datos, creamos e instanciamos el modelo.

In [17]:
# configuraciones
config = DistilBertConfig(num_labels=cfg03['num_labels'], seq_classif_dropout=0.5)


# cargar el modelo pre-entrenado disponible en Transformers
model03 = TFDistilBertForSequenceClassification.from_pretrained(cfg03['trained_model_name'], config=config)


# finalizar configuración del modelo
# se sugiere revisar documentación para más detalles sobre los diferentes hiper-parámetros
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)


# definir función loss. Debe cuidarse que sea coherente con la salida esperada del modelo (vector de num_labels elementos)
# y el formato de los ejemplos (vector one-hot de num_labels componentes para codificar las categorías)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)


# compilar el modelo, indicando otras métricas que se desee monitorear
# La métrica debe ser apropiada para el tipo de problema (clasificación binaria o multiclase)
model03.compile(optimizer=optimizer, loss=loss, metrics=['binary_accuracy'])


# imprimir sumario del modelo
model03.summary()

print('Done!')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use i

Model: "tf_distil_bert_for_sequence_classification_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  66362880  
_________________________________________________________________
pre_classifier (Dense)       multiple                  590592    
_________________________________________________________________
classifier (Dense)           multiple                  769       
_________________________________________________________________
dropout_79 (Dropout)         multiple                  0         
Total params: 66,954,241
Trainable params: 66,954,241
Non-trainable params: 0
_________________________________________________________________
Done!


Y finalmente entrenamos el modelo.

In [19]:
# configuraciones
checkpoints_dir = 'checkpoints'
trained_model_name = os.path.join(checkpoints_dir, 'distilbert-review')

epochs_max = 1
epochs_to_save = 1
batch_size = 16


# formatear los datos (tensores) de entrada de acuerdo a las opciones permitidas por TensorFlow 
train_inputs = { 'input_ids': train_encodings['input_ids'],
            'attention_mask': train_encodings['attention_mask']
         }

# ciclo de entrenamiento y guardar checkpoints
for epoch in tqdm(range(0, epochs_max, epochs_to_save)):
    print('Training model, epochs {0} - {1}'.format(epoch+1, epoch+epochs_to_save))
    
    # entrenar el modelo. Opcionalmente, se puede suministrar datos de validación => validation_data=(val_inputs,val_blabels_t )
    model03.fit(train_inputs, y=train_blabels_t, epochs=epochs_to_save, batch_size=batch_size)

    #model03.save_pretrained(trained_model_name + '-epochs-{0:03d}-{1:03d}'.format(epoch+1, epoch+epochs_to_save))
    #tokenizer.save_pretrained(trained_model_name + '-epochs-{0:03d}-{1:03d}'.format(epoch+1, epoch+epochs_to_save))

print('Done!')

  0%|          | 0/6 [00:00<?, ?it/s]

Training model, epochs 1 - 1
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


...

Training model, epochs 5 - 5
100/100 [==============================] - 95s 946ms/step - loss: 0.0448 - binary_accuracy: 0.9868


 83%|████████▎ | 5/6 [08:20<01:41, 101.29s/it]

Training model, epochs 6 - 6
100/100 [==============================] - 95s 946ms/step - loss: 0.0450 - binary_accuracy: 0.9874


100%|██████████| 6/6 [09:55<00:00, 99.22s/it]

Done!


### Construcción del ensemble

Como el stack de clasificadores se construirá de forma manual, es preciso realizar los siguientes pasos:

- utilizar los modelos base para predecir las instancias del conjunto de entrenamiento. En este caso en lugar de la categoría, recuperaremos la probabilidad asignada por el clasificador a que la instancia sea positiva.

- combinar las predicciones de cada modelo para obtener el conjunto de entrenamiento del metaclasificador.

- entrenar el metaclasificador

#### Obtener predicciones de los modelos base

Para entrenar el metaclasificador, utilizaremos la partición **train** previamente creada. Recordemos que hemos reservado la partición **eval** para evaluar cada modelo y compararlo respecto al  metaclasificador.

Ejecute las siguientes casillas para obtener las predicciones de los modelos base.

In [68]:
# predicciones modelo 1
m01_pscores = model01.predict_proba(train['Phrase'])[:,1]
print('Done!')

Done!


In [69]:
# predicciones modelo 2
# configuraciones
batch_size = 128 # puede indicar un valor menor para disminuir el consumo de memoria 
size = train.shape[0]


# predecir los datos de entrenamiento
m02_pred = []
m02_pscores = []
for i in tqdm(range(0, size, batch_size)):
    batch_text = train['Phrase'][i:i+batch_size].to_list()
    results = model02(batch_text, truncation=cfg02['truncation'])
    for pred in results:
      m02_pred.append(pred['label'].lower())
      m02_pscores.append(pred['score'] if pred['label']=='POSITIVE' else 1-pred['score'])
    
m02_pscores = np.asarray(m02_pscores)

print('Done!')

100%|██████████| 2/2 [02:20<00:00, 70.07s/it]

Done!


In [70]:
# predicciones modelo 3

# tokenizar
tokenizer = cfg03['tokenizer']
encodings = tokenizer(train['Phrase'].to_list(), truncation=True, padding='max_length', max_length=cfg03['max_length'], return_tensors=cfg03['framework'])


inputs = {'input_ids': encodings['input_ids'],
          'attention_mask': encodings['attention_mask'],
         }


# predecir los datos de prueba
m03_pscores = model03.predict(inputs)['logits'][:,0]

print('Done!')

In [81]:
# combinar predicciones ()
train_cmb =  pd.DataFrame({'m01_scores': m01_pscores, 'm02_scores': m02_pscores, 'm03_scores':m03_pscores, 'Sentiment':train['Sentiment']})


# separar entradas y salidas esperadas para satisfacer formato requerido por scikit-learn
X = train_cmb.loc[:,['m01_scores', 'm02_scores', 'm03_scores']].values
y = train_cmb['Sentiment']

print(train_cmb.head(5))
print('Done!')

ValueError: ignored

#### Instanciar y entrenar metaclasificador

Finalmente, podemos utilizar el nuevo conjunto de entrenamiento formado por la combinación de los predictores base para entrenar el metaclasificador. En este caso, utilizaremos la implementación de árboles de decisión en 

In [80]:
from sklearn.tree import DecisionTreeClassifier




# instanciar el clasificador
classifier = DecisionTreeClassifier(random_state=seed)


# entrenar el clasificador
classifier.fit(X, y)

array(['positive', 'positive', 'negative', 'positive', 'negative',
       'negative', 'positive', 'positive', 'negative', 'negative',
       'negative', 'negative', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'positive', 'negative', 'positive', 'positive', 'positive',
       'positive', 'negative', 'negative', 'positive', 'positive',
       'negative', 'negative', 'positive', 'negative', 'negative',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'positive', 'positive', 'positive', 'negative', 'positive',
       'positive', 'positive', 'positive', 'positive', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive',
       'negative', 'negative', 'negative', 'negative', 'negative',
       'positive', 'negative', 'positive', 'negative', 'positive',
       'negative', 'positive', 'positive', 'negative', 'positive',
       'positive', 'negative', 'positive', 'positive', 'negati

### Comparando resultados

Finalmente, realizaremos una comparación entre los resultados que alcanzan los modelos base y el ensemble. Para esto, utilizaremos la porción **val**.

#### Evaluación Modelo 1

#### Evaluación Modelo 2

#### Evaluación Modelo 3

#### Evaluación Ensemble